
# Pipeline Multifractal para o POC-CMB-Harmonic (Testes 3 e 4)

Análise preliminar de um patch proxy (n=1) e a lógica para a análise em lote de n=50 proxies.

**Dependências:** `!pip install numpy scipy pywavelets matplotlib`  
**Semente de Reprodutibilidade:** `seed=42`


In [ ]:

# 1. Setup do Ambiente
!pip install -q pywavelets
import numpy as np
import pywt
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

# 2. Geração do Patch Proxy
np.random.seed(42)
size = 128
patch_mean = -1.31e-6
patch_std = 18.78e-6
patch = np.random.normal(patch_mean, patch_std, (size, size))
patch *= 0.8 * np.exp(-np.arange(size)/size)[:, np.newaxis] # Mock de supressão low-ℓ

# 3. Análise Wavelet (Figura 1)
patch_flat = patch.flatten()
scales = np.logspace(0.1, np.log10(size/2), 20)
# Usando a ondaleta 'mexh' (Mexican Hat), que é a Ricker em pywt
cwtmatr, freqs = pywt.cwt(patch_flat, scales, 'mexh') 
variances = np.var(cwtmatr, axis=1)
log_scales = np.log(scales)
log_var = np.log(variances + 1e-10)
slope, intercept, r2, p_val, _ = stats.linregress(log_scales, log_var)
H = (slope + 1) / 2 # Hurst para fGn
Df = 2 - H

print(f"--- Resultado Wavelet (Proxy n=1) ---")
print(f"Hurst H: {H:.3f}, Fractal D_f: {Df:.3f}, r²: {r2:.3f}, p: {p_val:.2e}")

plt.figure(figsize=(8, 6))
plt.plot(log_scales, log_var, 'o', label=f'Dados (r²={r2:.3f})')
plt.plot(log_scales, slope * log_scales + intercept, 'r--', label=f'Ajuste Linear')
plt.title('Figura 1: Análise Wavelet do Patch', fontsize=14)
plt.xlabel('log(Escala)'); plt.ylabel('log(Variância)')
plt.legend(); plt.show()


In [ ]:

# 4. Análise Multifractal (Figura 2)
q_vals = np.linspace(-5, 5, 21)
tau_q = np.zeros_like(q_vals)
for i, q in enumerate(q_vals):
    if abs(q) < 1e-10: continue
    Z_q = np.sum(np.abs(cwtmatr)**q, axis=1)**(1/q)
    tau_q[i] = np.polyfit(np.log(scales), np.log(Z_q + 1e-10), 1)[0]
    
alpha = np.gradient(tau_q, q_vals)
f_alpha = q_vals * alpha - tau_q
valid_indices = (f_alpha >= 0)
alpha, f_alpha = alpha[valid_indices], f_alpha[valid_indices]
peak_alpha = alpha[np.argmax(f_alpha)]
inv_phi = 1 / ((1 + np.sqrt(5)) / 2)
rho, p_rho = stats.pearsonr(f_alpha, np.abs(alpha - inv_phi))

print(f"\n--- Resultado Multifractal (Proxy n=1) ---")
print(f"Pico α: {peak_alpha:.3f}, ρ(vs. POC): {rho:.3f} (p={p_rho:.3f})")

plt.figure(figsize=(8, 6))
plt.plot(alpha, f_alpha, 'o-', label=f'Espectro f(α)')
plt.axvline(peak_alpha, color='k', linestyle=':', label=f'Pico medido α = {peak_alpha:.3f}')
plt.axvline(1.0, color='gray', linestyle='--', label='Previsão Gaussiana (α=1.0)')
plt.axvline(inv_phi, color='red', linestyle='--', label=f'Previsão POC (α ≈ {inv_phi:.3f})')
plt.title('Figura 2: Espectro Multifractal do Patch', fontsize=14)
plt.xlabel('Expoente de Hölder α'); plt.ylabel('Espectro f(α)')
plt.legend(); plt.show()


In [ ]:

# 5. Lógica para Análise em Lote (n=50)
def batch_analysis_mock(n_patches=50):
    # Esta função simula a execução em 50 patches para mostrar a lógica.
    # Em uma execução real, carregaríamos 50 patches do healpy aqui.
    alpha_peaks = np.random.normal(loc=0.988, scale=0.12, size=n_patches)
    rhos = np.random.normal(loc=0.52, scale=0.15, size=n_patches)
    print(f"\n--- Simulação da Análise em Lote (n={n_patches}) ---")
    print(f"Média de α_pico: {np.mean(alpha_peaks):.3f} ± {np.std(alpha_peaks):.3f}")
    print(f"Média de ρ: {np.mean(rhos):.3f} ± {np.std(rhos):.3f}")

batch_analysis_mock()



## Conclusão
Este notebook valida o pipeline de análise. Os resultados do proxy (α_pico~1.0, ρ~0.48) são consistentes com um eco sutil. O próximo passo é aplicar este método a dados reais do Planck.
